## 1. 라이브러리 임포트

In [5]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

sns.set(style="whitegrid")

## 2. CSV (또는 Parquet 등) 로드

In [ ]:
FILE_PATH = "../data/raw/event_log.csv"
df = pd.read_csv(FILE_PATH)

# 3. 기본 구조 확인

In [ ]:
print("=== DataFrame Head ===")
display(df.head())

print("=== Info ===")
df.info()

print("=== Null Counts ===")
print(df.isnull().sum())

print("=== Describe (All) ===")
display(df.describe(include="all"))

## 4. 주요 컬럼 분포 살펴보기

### (1) Event Category 분포

In [ ]:
print("=== Event Category 분포 ===")
display(df["Event Category"].value_counts())

plt.figure(figsize=(6,4))
sns.countplot(data=df, x="Event Category", order=df["Event Category"].value_counts().index)
plt.xticks(rotation=45)
plt.title("Event Category Distribution")
plt.xlabel("Event Category")
plt.ylabel("Count")
plt.show()

### (2) Event Date, Event Datetime 처리

In [ ]:
#     Event Date, Event Datetime이 문자열일 수 있으니 datetime으로 변환
df["Event Date"] = pd.to_datetime(df["Event Date"], errors="coerce")  # yyyy-mm-dd
df["Event Datetime"] = pd.to_datetime(df["Event Datetime"], errors="coerce") # yyyy-mm-ddTHH:MM:SS+09:00

# Date가 정상 변환되었는지 확인
print("Date range:", df["Event Date"].min(), " ~ ", df["Event Date"].max())
print("Datetime range:", df["Event Datetime"].min(), " ~ ", df["Event Datetime"].max())

# 날짜별 이벤트 추이
df_date_counts = df.groupby("Event Date")["Hashed User ID"].count().reset_index()
df_date_counts.columns = ["Event Date", "Count"]

plt.figure(figsize=(8,4))
sns.barplot(data=df_date_counts, x="Event Date", y="Count")
plt.xticks(rotation=45)
plt.title("Events by Date")
plt.show()

# 시간대별 이벤트 추이 (Event Datetime에서 시(hour) 추출)
df["Event Hour"] = df["Event Datetime"].dt.hour
plt.figure(figsize=(6,4))
sns.countplot(data=df, x="Event Hour")
plt.title("Events by Hour of Day")
plt.xlabel("Hour")
plt.ylabel("Count")
plt.show()

### (3) Hashed User ID 관련

In [ ]:
print("=== Unique Users ===")
unique_users = df["Hashed User ID"].nunique()
print(f"Unique Hashed User IDs: {unique_users}")

# 사용자별 이벤트 건수 분포
user_counts = df["Hashed User ID"].value_counts()
plt.figure(figsize=(6,4))
plt.hist(user_counts, bins=20, edgecolor="black")
plt.title("Distribution of Event Counts per User")
plt.xlabel("Number of Events")
plt.ylabel("Count of Users")
plt.show()

## 5. Semantic Event Properties (JSON) 파싱

In [ ]:
#    각 row마다 JSON 구조가 다를 수 있으므로, 공통 Key를 확인 후 처리

# 예: "products" 라는 key 아래에 상품정보들이 배열로 들어있는 케이스 (샘플처럼)
#     {"products":[{"name":"...", "price":..., "productID":"...", "quantity":...}, ...], ...}

def parse_json_column(json_str):
    """
    JSON 문자열을 파이썬 dict로 변환.
    에러 발생 시 빈 dict 반환.
    """
    try:
        return json.loads(json_str)
    except:
        return {}

df["parsed_props"] = df["Semantic Event Properties"].apply(parse_json_column)

# (1) products 리스트 추출 (없으면 빈 리스트 반환)
df["products"] = df["parsed_props"].apply(lambda x: x.get("products", []))

# (2) explode를 이용해 products 배열을 행으로 펼치기
df_exploded = df.explode("products", ignore_index=True)

# products가 없었던 행들은 NaN이 됨
# product 정보 열 추출
df_exploded["product_name"] = df_exploded["products"].apply(lambda x: x.get("name", "") if isinstance(x, dict) else None)
df_exploded["product_price"] = df_exploded["products"].apply(lambda x: x.get("price", np.nan) if isinstance(x, dict) else None)
df_exploded["product_id"] = df_exploded["products"].apply(lambda x: x.get("productID", "") if isinstance(x, dict) else None)
df_exploded["product_quantity"] = df_exploded["products"].apply(lambda x: x.get("quantity", np.nan) if isinstance(x, dict) else None)

print("=== Exploded DataFrame Head ===")
display(df_exploded.head(10))

## 6. 상품 정보 분석 예시
- 주문(event category가 "Order Complete" 등) 중심으로만 분석할 수도 있음

### (1) 상품별 판매량(또는 이벤트 발생량)

In [ ]:
product_quantity_sum = df_exploded.groupby("product_id")["product_quantity"].sum().reset_index()
product_quantity_sum.columns = ["product_id", "total_quantity"]
product_quantity_sum = product_quantity_sum.sort_values("total_quantity", ascending=False)

print("=== Top 10 products by total quantity ===")
display(product_quantity_sum.head(10))

### (2) 상품별 평균 가격

In [ ]:
product_price_mean = df_exploded.groupby("product_id")["product_price"].mean().reset_index()
product_price_mean.columns = ["product_id", "avg_price"]
product_price_mean = product_price_mean.sort_values("avg_price", ascending=False)

print("=== Top 10 products by average price ===")
display(product_price_mean.head(10))

# 시각화 예시: 상위 몇 개 상품의 누적 판매량
topN = 10
topN_products = product_quantity_sum.head(topN)
plt.figure(figsize=(8,4))
sns.barplot(data=topN_products, x="product_id", y="total_quantity", edgecolor="black")
plt.xticks(rotation=45)
plt.title(f"Top {topN} Products by Total Quantity")
plt.show()

## 7. Event Category 별로 상품이 어떻게 등장하는지(간단 예시)

In [ ]:
cat_group = df_exploded.groupby(["Event Category"])["product_id"].nunique().reset_index()
cat_group.columns = ["Event Category", "unique_products"]
print("=== Unique products per Event Category ===")
display(cat_group)

## 8. 요약

In [ ]:
print("=== Quick Summary ===")
print(f"Total Rows: {len(df)} (original) -> {len(df_exploded)} (exploded)")
print(f"Unique Users: {unique_users}")
print(f"Unique Products (after explode): {df_exploded['product_id'].nunique()}")
print("Date range:", df['Event Date'].min(), " to ", df['Event Date'].max())
print("Event Category counts:")
print(df['Event Category'].value_counts())